In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from app.api.handlers.cloudinary import CloudinaryHandler

handler = CloudinaryHandler.from_env()

assets = handler.pull_audio_details()
print(f"Found {len(assets)} assets")

Found 10 assets
